In [13]:
import json 
from tqdm import tqdm
from collections import Counter
import re

In [2]:
path = '/data/home/wikidata-dump/'

In [7]:
# type_dict = {}
type_count = Counter()

with open(path+'out-types.txt', 'r') as f:
    for line in tqdm(f):
        line = line.split(":")
        num, content = line[0], line[1]
        type = content.split(",")[0].replace('\"', '')
        # type_dict[int(num)] = type
        type_count[type] += 1


107674454it [01:40, 1076049.42it/s]


In [8]:
type_count

Counter({'item': 107662729, 'property': 11725})

In [9]:
del type_count

In [12]:
!ls $path

latest-all.json.bz2	     out-instances_of.txt  out.txt
out-descriptions-labels.txt  out-subclass_of.txt   out-types.txt


In [45]:
# type_dict = {}
instance_count = Counter()

with open(path+'out-instances_of.txt', 'r') as f:
    for line in tqdm(f):

        line = line.split(":")
        num, content = line[0], ":".join(line[1:])

        content = re.findall('(?<={"mainsnak":{"snaktype":"value","property":"P31").*?(?=})', content)

        for instance_str in content:
            instance = re.findall('(?<="id":").*?(?=")', instance_str)[0]

            instance_count[instance] += 1


17919it [00:00, 90250.13it/s]

105103875it [18:39, 93870.65it/s] 


In [49]:
instance_count.most_common(30)

[('Q13442814', 41490382),
 ('Q5', 11120920),
 ('Q4167836', 5320827),
 ('Q16521', 3717453),
 ('Q523', 3291638),
 ('Q318', 2101693),
 ('Q4167410', 1476083),
 ('Q113145171', 1262002),
 ('Q7187', 1071741),
 ('Q8054', 1002075),
 ('Q3305213', 926355),
 ('Q11266439', 820059),
 ('Q79007', 671630),
 ('Q13433827', 626667),
 ('Q101352', 601548),
 ('Q13100073', 588577),
 ('Q486972', 574475),
 ('Q8502', 523113),
 ('Q2668072', 502972),
 ('Q3331189', 426744),
 ('Q4022', 415326),
 ('Q30612', 391756),
 ('Q13406463', 359078),
 ('Q1931185', 354111),
 ('Q54050', 319967),
 ('Q11424', 304858),
 ('Q482994', 293138),
 ('Q1457376', 292566),
 ('Q17633526', 288845),
 ('Q23397', 287125)]

In [47]:
len(instance_count)

94477

In [68]:
import requests
import time

def get_entity_name_from_wikidata_id(entity_id):
    # Define the Wikidata endpoint URL
    endpoint_url = "https://query.wikidata.org/sparql"

    # Construct the SPARQL query to get the label (name) of the entity
    query = f"""
        SELECT ?entityLabel WHERE {{
            wd:{entity_id} rdfs:label ?entityLabel.
            FILTER(LANG(?entityLabel) = "en")
        }}
    """

    # Set the SPARQL query parameters
    params = {
        "format": "json",
        "query": query
    }
    response = requests.get(endpoint_url, params=params)

    try:
        # Send the GET request to the Wikidata endpoint
        
        data = response.json()
        
        # Parse the JSON response to extract the entity name
        if "results" in data and "bindings" in data["results"]:
            bindings = data["results"]["bindings"]
            if bindings:
                # Get the value of the entityLabel from the first result
                entity_name = bindings[0]["entityLabel"]["value"]
                return entity_name
            
    except Exception as e:
        print(f"Error occurred: {response}")
        
    
    return None

id2name = {}
for item in tqdm(instance_count.most_common(100)):
    item_name = get_entity_name_from_wikidata_id(item[0])
    id2name[item[0]] = item_name
    time.sleep(1)

 52%|█████▏    | 52/100 [01:02<01:00,  1.25s/it]

Error occurred: <Response [429]>


 55%|█████▌    | 55/100 [01:06<00:54,  1.22s/it]

Error occurred: <Response [429]>


 68%|██████▊   | 68/100 [01:22<00:40,  1.28s/it]

Error occurred: <Response [429]>


100%|██████████| 100/100 [02:03<00:00,  1.23s/it]


In [69]:
id2name

{'Q13442814': 'scholarly article',
 'Q5': 'human',
 'Q4167836': 'Wikimedia category',
 'Q16521': 'taxon',
 'Q523': 'star',
 'Q318': 'galaxy',
 'Q4167410': 'Wikimedia disambiguation page',
 'Q113145171': 'type of chemical entity',
 'Q7187': 'gene',
 'Q8054': 'protein',
 'Q3305213': 'painting',
 'Q11266439': 'Wikimedia template',
 'Q79007': 'street',
 'Q13433827': 'encyclopedia article',
 'Q101352': 'family name',
 'Q13100073': 'village',
 'Q486972': 'human settlement',
 'Q8502': 'mountain',
 'Q2668072': 'collection',
 'Q3331189': 'version, edition or translation',
 'Q4022': 'river',
 'Q30612': 'clinical trial',
 'Q13406463': 'Wikimedia list article',
 'Q1931185': 'astronomical radio source',
 'Q54050': 'hill',
 'Q11424': 'film',
 'Q482994': 'album',
 'Q1457376': 'eclipsing binary star',
 'Q17633526': 'Wikinews article',
 'Q23397': 'lake',
 'Q39614': 'cemetery',
 'Q2247863': 'high proper-motion star',
 'Q16970': 'church building',
 'Q3863': 'asteroid',
 'Q7725634': 'literary work',
 'Q41

In [70]:
for item, freq in instance_count.most_common(100):
    print(id2name[item], freq)

scholarly article 41490382
human 11120920
Wikimedia category 5320827
taxon 3717453
star 3291638
galaxy 2101693
Wikimedia disambiguation page 1476083
type of chemical entity 1262002
gene 1071741
protein 1002075
painting 926355
Wikimedia template 820059
street 671630
encyclopedia article 626667
family name 601548
village 588577
human settlement 574475
mountain 523113
collection 502972
version, edition or translation 426744
river 415326
clinical trial 391756
Wikimedia list article 359078
astronomical radio source 354111
hill 319967
film 304858
album 293138
eclipsing binary star 292566
Wikinews article 288845
lake 287125
cemetery 286865
high proper-motion star 274321
church building 264457
asteroid 247959
literary work 247185
building 244923
United States patent 231416
village 231231
business 220749
house 217742
architectural structure 210901
stream 187306
biographical article 180460
watercourse 179535
state school 178856
sports season 173168
quasar 168038
island 163634
dictionary entry 15

In [72]:
# type_dict = {}
subclass_count = Counter()

with open(path+'out-subclass_of.txt', 'r') as f:
    for line in tqdm(f):

        line = line.split(":")
        num, content = line[0], ":".join(line[1:])

        content = re.findall('(?<={"mainsnak":{"snaktype":"value","property":"P279").*?(?=})', content)

        for subclass_str in content:
            subclass = re.findall('(?<="id":").*?(?=")', subclass_str)[0]

            subclass_count[subclass] += 1


4118820it [00:43, 94590.50it/s] 


In [73]:
len(subclass_count)

239849

In [74]:
id2name = {}

for item in tqdm(subclass_count.most_common(100)):
    item_name = get_entity_name_from_wikidata_id(item[0])
    id2name[item[0]] = item_name
    time.sleep(1)

  3%|▎         | 3/100 [00:03<02:02,  1.27s/it]

Error occurred: <Response [429]>


  5%|▌         | 5/100 [00:06<01:58,  1.25s/it]

Error occurred: <Response [429]>


100%|██████████| 100/100 [02:07<00:00,  1.28s/it]


In [76]:
for item, freq in subclass_count.most_common(100):
    print(id2name[item], freq)


chemical compound 1146000
protein 769218
protein-coding gene 514163
None 453776
pseudogene 44672
None 40214
municipal councillor 40211
non-coding RNA 37279
CN 11955
transfer RNA 8798
pseudogenic transcript 8788
mayor of a place in Spain 8022
alkaloid 5596
minister 5181
aircraft 4727
protein-containing complex 4230
biogenic cyclopeptide 4120
oleanane triterpenoid 4038
flavone 3471
small nucleolar RNA 3392
macrolides 3203
mayor of a place in Romania 3186
2,6-dimethyloctane monoterpenoid 2935
tender locomotive 2689
per- and polyfluoroalkyl substances 2670
food 2591
Italian wine 2517
mayor 2459
council member 2369
eudesmane sesquiterpenoid 2300
germacrane sesquiterpenoid 2201
electric multiple unit 2171
fatty alcohol 2124
biogenic alkylphenol 2035
guaiane sesquiterpenoid 1934
ribosomal RNA 1884
mobile phone 1826
biogenic acyclic ester 1815
tennis tournament 1749
transport protein 1738
labdanes 1734
tank locomotive 1698
motor car 1670
Dutch municipal councillor 1658
mayor of a place in the 